## Preprocessing & Feature Engineering

**Objectif :** Transformer les données brutes (validées lors de l'EDA) en un dataset mathématiquement exploitable par les algorithmes de Machine Learning.

**La Règle d'Or respectée ici : L'immuabilité des données.**
Les fichiers sources du dossier `data/raw/` ne sont jamais modifiés. Les transformations se font en mémoire vive, et le résultat sera sauvegardé dans un nouvel environnement (`data/processed/`).

### 1. Chargement des Données

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder

print("📥 1. Chargement des données brutes...")
df_p = pd.read_csv("../data/raw/patients.csv", parse_dates=["date_et_heure_admission"])
df_rh = pd.read_csv("../data/raw/personnel.csv", parse_dates=["date_heure_prise_poste"])
df_mat = pd.read_csv("../data/raw/materiel.csv", parse_dates=["date_heure_inventaire"])

print(f"Dimensions initiales - Patients: {df_p.shape}")
df_p.head(3) # Affiche les 3 premières lignes pour vérifier

📥 1. Chargement des données brutes...
Dimensions initiales - Patients: (852492, 9)


,ID_Patient,age,sexe,motif_admission,ccmu,duree_hospitalisation,date_et_heure_admission,service_admission,issue
0,4e2bb4bc,49,F,Fievre Inexpliquee,3,34.8,2018-01-01 14:03:00,Chirurgie_Cardio,Transfert
1,1b90a704,53,M,Douleur Thoracique,2,4.2,2018-01-01 10:28:00,Urgences,Retour Domicile
2,8a6feedc,74,F,Douleur Thoracique,4,51.5,2018-01-01 11:52:00,Medecine_Interne,Transfert


### 2. Nettoyage des Anomalies (Data Cleaning)

L'Analyse Exploratoire (EDA) a mis en évidence deux bruits statistiques générés par le simulateur :
1. **Âges aberrants :** La queue de la distribution normale a généré des âges < 0 ou > 105 ans.
2. **Incohérence Métier :** Présence de patients de sexe Masculin en Gynécologie-Obstétrique.

**Action :** Filtrage de ces anomalies pour garantir la pertinence clinique du modèle.

In [4]:
initial_len = len(df_p)

# 1. Âges aberrants (On garde entre 0 et 105 ans)
df_p = df_p[(df_p['age'] >= 0) & (df_p['age'] <= 105)]

# 2. Incohérence Sexe/Service (Pas d'hommes en Gynéco)
incoherent_mask = (df_p['service_admission'] == "Gyneco_Obstetrique") & (df_p['sexe'] == "M")
df_p = df_p[~incoherent_mask]

print(f"✅ Lignes supprimées : {initial_len - len(df_p)}")
print(f"Nouvelles dimensions : {df_p.shape}")

✅ Lignes supprimées : 0
Nouvelles dimensions : (799657, 16)


## 3. Feature Engineering : Le Temps Cyclique

Les algorithmes classiques considèrent les nombres de façon linéaire (23 est très loin de 0). Or, dans un hôpital, l'heure `23:59` est temporellement collée à `00:01`. 

**Action :** Nous transformons les heures et les mois en coordonnées trigonométriques $sin$ et $cos$. Cela permet à l'IA de "comprendre" les cycles naturels du temps.

In [6]:
print("⚙️ 3. Création des variables temporelles")

# Variables Temporelles 
df_p['date'] = df_p['date_et_heure_admission'].dt.date
df_p['hour'] = df_p['date_et_heure_admission'].dt.hour
df_p['month'] = df_p['date_et_heure_admission'].dt.month
df_p['weekday'] = df_p['date_et_heure_admission'].dt.weekday

# Transformation Cyclique (Sin/Cos)
df_p['sin_hour'] = np.sin(2 * np.pi * df_p['hour'] / 24)
df_p['cos_hour'] = np.cos(2 * np.pi * df_p['hour'] / 24)
df_p['est_weekend'] = df_p['weekday'].apply(lambda x: 1 if x >= 5 else 0)

⚙️ 3. Création des variables temporelles


### 4. Feature Engineering : L'Inertie du Système (Lag Features)

La saturation hospitalière possède une forte inertie. Si l'hôpital est saturé à J-1, le risque est maximal pour le jour J.
**Action :** Création d'une variable de "mémoire" (`patients_hier`) pour donner le contexte récent au modèle.

In [7]:
print("⏳ Calcul de l'historique récent (Lag Features)...")

daily_flux = df_p.groupby(['date', 'service_admission']).size().reset_index(name='patients_jour')
daily_flux['patients_hier'] = daily_flux.groupby('service_admission')['patients_jour'].shift(1)

# Fusion avec le dataset principal
df_p = pd.merge(df_p, daily_flux[['date', 'service_admission', 'patients_hier']], 
                     on=['date', 'service_admission'], how='left')

# Imputation par la moyenne pour le tout premier jour historique
df_p['patients_hier'] = df_p['patients_hier'].fillna(df_p['patients_hier'].mean())

⏳ Calcul de l'historique récent (Lag Features)...


### 5. Intégration des Ressources et Correction Métier

Nous agrégeons ici l'Offre de Soins (RH et Matériel) au niveau journalier pour la confronter à la Demande (Patients).
*Note de Data Quality :* Suite à l'EDA, la nomenclature de panne matériel a été corrigée (passage de "Panne" à "Indisponible").

In [8]:
# Préparation RH
df_rh['date'] = df_rh['date_heure_prise_poste'].dt.date
rh_agg = df_rh.groupby(['date', 'service']).agg({'effectif_present': 'mean', 'taux_absenteisme': 'mean'}).reset_index()

# Préparation Matériel (Avec la correction du mot-clé "Indisponible")
df_mat['date'] = df_mat['date_heure_inventaire'].dt.date
mat_agg = df_mat.groupby(['date', 'services']).agg({
    'nbre_lits_dispos': 'min',
    'equipements_disponibles': lambda x: 1 if "Indisponible" in str(x.values) else 0 
}).reset_index().rename(columns={'services': 'service', 'equipements_disponibles': 'panne_materiel'})

# Fusion Finale
df_merged = pd.merge(df_p, rh_agg, left_on=['date', 'service_admission'], right_on=['date', 'service'], how='left')
df_merged = pd.merge(df_merged, mat_agg, on=['date', 'service'], how='left')

### 6. Définition de la Cible (Target) et Export

L'objectif de l'IA est de prédire une saturation (`EST_SATURE`).
**Règle métier retenue :** Un hôpital est considéré en état de saturation critique lorsque la durée de séjour d'un patient dépasse le 75ème centile de son service (preuve visuelle issue de la "longue traîne" de l'EDA).

Nous encodons ensuite les variables textuelles (Sexe, Service) en numériques, puis nous exportons le dataset finalisé.

In [9]:
# Cible : 1 si LOS > 75ème centile du service
seuil_tension = df_merged.groupby('service')['duree_hospitalisation'].transform(lambda x: x.quantile(0.75))
df_merged['EST_SATURE'] = (df_merged['duree_hospitalisation'] > seuil_tension).astype(int)

# Encodage
df_merged['sexe_enc'] = df_merged['sexe'].apply(lambda x: 1 if x == 'F' else 0)
le = LabelEncoder()
df_merged['service_enc'] = le.fit_transform(df_merged['service_admission'])

# Nettoyage des colonnes inutiles pour le ML
cols_to_drop = ['ID_Patient', 'date_et_heure_admission', 'issue', 'date', 'service', 
                'motif_admission', 'service_admission', 'sexe']
df_final = df_merged.drop(columns=cols_to_drop, errors='ignore').fillna(0)

# Sauvegarde dans le dossier PROCESSED
os.makedirs("../data/processed", exist_ok=True)
output_path = "../data/processed/train_data.csv"
df_final.to_csv(output_path, index=False)

print(f"✅ Preprocessing Terminé !")
print(f"📊 Pourcentage de situations de saturation (Classe 1) : {df_final['EST_SATURE'].mean() * 100:.2f}%")
print(f"💾 Fichier exporté avec succès : {output_path}")

✅ Preprocessing Terminé !
📊 Pourcentage de situations de saturation (Classe 1) : 24.86%
💾 Fichier exporté avec succès : ../data/processed/train_data.csv
